<h1> 2d. Distributed training and monitoring </h1>

In this notebook, we refactor to call ```train_and_evaluate``` instead of hand-coding our ML pipeline. This allows us to carry out evaluation as part of our training loop instead of as a separate step. It also adds in failure-handling that is necessary for distributed training capabilities.

We also use TensorBoard to monitor the training.

In [1]:
# import datalab.bigquery as bq
from google.cloud import bigquery as bq
import tensorflow as tf
import numpy as np
import shutil
# from google.datalab.ml import TensorBoard
print('google-cloud-bigquery:', bq.__version__)
print('tensorlfow version:',tf.__version__)

google-cloud-bigquery: 1.5.0
tensorlfow version: 1.10.0


/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<h2> Input </h2>

Read data created in Lab1a, but this time make it more general, so that we are reading in batches.  Instead of using Pandas, we will use add a filename queue to the TensorFlow graph.

In [2]:
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [[0.0], [-74.0], [40.0], [-74.0], [40.7], [1.0], ['nokey']]

def read_dataset(filename, mode, batch_size = 512):
    def _input_fn():
        def decode_csv(value_column):
            columns = tf.decode_csv(value_column, record_defaults = DEFAULTS)
            features = dict(zip(CSV_COLUMNS, columns))
            label = features.pop(LABEL_COLUMN)
            return features, label

        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        filenames_dataset = tf.data.Dataset.list_files(filename)
        # Read lines from text files
        textlines_dataset = filenames_dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        dataset = textlines_dataset.map(decode_csv)

        # Note:
        # use tf.data.Dataset.flat_map to apply one to many transformations (here: filename -> text lines)
        # use tf.data.Dataset.map      to apply one to one  transformations (here: text line -> feature list)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
    
        return dataset.make_one_shot_iterator().get_next()
    return _input_fn
    

def get_train():
    return read_dataset('./taxi-train.csv', mode = tf.estimator.ModeKeys.TRAIN)

def get_valid():
    return read_dataset('./taxi-valid.csv', mode = tf.estimator.ModeKeys.EVAL)

def get_test():
    return read_dataset('./taxi-test.csv', mode = tf.estimator.ModeKeys.EVAL)

<h2> Create features out of input data </h2>

For now, pass these through.  (same as previous lab)

In [3]:
# define feature columns.
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('pickuplon'),
    tf.feature_column.numeric_column('pickuplat'),
    tf.feature_column.numeric_column('dropofflat'),
    tf.feature_column.numeric_column('dropofflon'),
    tf.feature_column.numeric_column('passengers'),
]

def add_more_features(feats):
  # Nothing to add (yet!)
  return feats

feature_cols = add_more_features(INPUT_COLUMNS)

<h2> Serving input function </h2>

In [4]:
# Defines the expected shape of the JSON feed that the model
# will receive once deployed behind a REST API in production.
def serving_input_fn():
    feature_placeholders = {
                            'pickuplon' : tf.placeholder(tf.float32, [None]),
                            'pickuplat' : tf.placeholder(tf.float32, [None]),
                            'dropofflat' : tf.placeholder(tf.float32, [None]),
                            'dropofflon' : tf.placeholder(tf.float32, [None]),
                            'passengers' : tf.placeholder(tf.float32, [None]),
                            }
    # You can transforma data here from the input format to the format expected by your model.
    features = feature_placeholders # no transformation needed
    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

<h2> tf.estimator.train_and_evaluate </h2>

In [5]:
def train_and_evaluate(output_dir, num_train_steps):
    estimator = tf.estimator.LinearRegressor(
                                model_dir = output_dir,
                                feature_columns = feature_cols)
    train_spec=tf.estimator.TrainSpec(
                                input_fn = read_dataset('./taxi-train.csv', mode = tf.estimator.ModeKeys.TRAIN),
                                max_steps = num_train_steps)
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec=tf.estimator.EvalSpec(
                                input_fn = read_dataset('./taxi-valid.csv', mode = tf.estimator.ModeKeys.EVAL),
                                steps = None,
                                start_delay_secs = 1, # start evaluating after N seconds
                                throttle_secs = 10,  # evaluate every N seconds
                                exporters = exporter)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

<h2> Monitoring with TensorBoard </h2>
<br/>
Use "refresh" in Tensorboard during training to see progress.

In [6]:
import os
os.path.abspath(os.path.dirname('__file__'))

'/home/chet/Desktop/Alfred/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow'

In [7]:
#这个directory要看你的jupyter notenotebook在哪里打开的啊...
# OUTDIR = 'taxi_trained' 

OUTDIR = os.path.realpath(os.path.join(os.path.abspath(os.path.dirname('__file__')),'taxi_trained'))
print(OUTDIR)

/home/chet/Desktop/Alfred/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow/taxi_trained


In [8]:
!tensorboard --logdir=OUTDIR

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
E0816 16:16:27.289432 MainThread program.py:267] TensorBoard attempted to bind to port 6006, but it was already in use
E0816 16:16:27.289432 139680802428224 program.py:267] TensorBoard attempted to bind to port 6006, but it was already in use
TensorBoard attempted to bind to port 6006, but it was already in use


In [ ]:
# OUTDIR = 'taxi_trained'
# TensorBoard().start(OUTDIR)

<h2>Run training</h2>

In [9]:
# Run training    
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
train_and_evaluate(OUTDIR, num_train_steps = 2000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/chet/Desktop/Alfred/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow/taxi_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7feab9118978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arg

<h4> You can now shut Tensorboard down </h4>

In [ ]:
# to list Tensorboard instances
TensorBoard().list()

In [ ]:
# to stop TensorBoard fill the correct pid below
TensorBoard().stop(27855)
print("Stopped Tensorboard")

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License